# Day 7: No Space Left On Device
https://adventofcode.com/2022/day/7

In [2]:
let lines: Vec<String> = std::fs::read_to_string("input/day07.txt")
    .expect("Could not read file")
    .lines()
    .map(|line| line.to_owned())
    .collect();

In [3]:
let example: Vec<String> = "$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k".lines().map(|line| line.to_owned()).collect();

In [4]:
#[derive(Debug)]
enum Item {
    File(String, usize),
    Directory(String)
}

In [5]:
#[derive(Debug)]
enum Command {
    Cd(String),
    Ls(Vec<Item>)
}

In [6]:
fn parse_input(lines: &[String]) -> Vec<Command> {
    let mut commands: Vec<Command> =  Vec::new();
    
    for line in lines {
        match line.split_whitespace().collect::<Vec<_>>().as_slice() {
            ["$", "ls"] => commands.push(Command::Ls(Vec::new())),
            ["$", "cd", dir_name] => commands.push(Command::Cd(dir_name.to_string())),
            [prefix, item_name] => {
                let item = if *prefix == "dir" {
                    Item::Directory(item_name.to_string())
                } else {
                    Item::File(item_name.to_string(), prefix.parse::<usize>().unwrap())
                };
                    
                match commands.last_mut() {
                    Some(Command::Ls(output)) => output.push(item),
                    _ => panic!("cannot append output to ls command")
                }
            }
            _ => panic!("error")
        }
    }
    
    commands
}

In [7]:
use std::collections::HashMap;

In [8]:
fn ls_results_per_path(commands: Vec<Command>) -> HashMap<Vec<String>, Vec<Item>> {
    let mut result: HashMap<Vec<String>, Vec<Item>> = HashMap::new(); 
    let mut path: Vec<String> = Vec::new();
    
    for command in commands {
        match command {
            Command::Cd(dir) => {
                match dir.as_str() {
                    "/" => path.clear(),
                    ".." => {
                        path.pop();
                    }
                    subdir => path.push(subdir.to_owned())
                }
            }
            Command::Ls(items) => {
                result.insert(path.clone(), items);
            }
        }
    }
    
    result
}

In [9]:
#[derive(Debug)]
struct Directory {
    files: HashMap<String, usize>,
    directories: HashMap<String, Directory>
}

In [10]:
fn directory_for_path(items_per_path: &mut HashMap<Vec<String>, Vec<Item>>, path: &mut Vec<String>) -> Directory {
    let mut files: HashMap<String, usize> = HashMap::new();
    let mut directories: HashMap<String, Directory> = HashMap::new();

    let items = items_per_path.remove(path).unwrap();
    for item in items {
        match item {
            Item::File(name, size) => {
                files.insert(name, size);
            },
            Item::Directory(name) => {
                path.push(name);
                let directory = directory_for_path(items_per_path, path);
                let name = path.pop().unwrap();
                directories.insert(name, directory);
            }
        }
    }
    
    Directory {
        files,
        directories
    }
}

In [11]:
fn file_system(items_per_path: HashMap<Vec<String>, Vec<Item>>) -> Directory {
    let mut items_per_path = items_per_path;  // make mutable
    directory_for_path(&mut items_per_path, &mut Vec::new())
}

In [12]:
file_system(ls_results_per_path(parse_input(&example)))

Directory { files: {"c.dat": 8504156, "b.txt": 14848514}, directories: {"d": Directory { files: {"j": 4060174, "d.ext": 5626152, "d.log": 8033020, "k": 7214296}, directories: {} }, "a": Directory { files: {"h.lst": 62596, "f": 29116, "g": 2557}, directories: {"e": Directory { files: {"i": 584}, directories: {} }} }} }

In [14]:
impl Directory {
    fn size(self: &Self) -> usize {
        self.files.values().sum::<usize>() + 
        self.directories.values().map(|dir| dir.size()).sum::<usize>()
    }
    
    fn for_each_subdirectory<F>(self: &Self, f: &mut F)
        where F: FnMut(&Directory) {
            f(self);
            for dir in self.directories.values() {
                dir.for_each_subdirectory(f);
            }
    }
}

In [20]:
file_system(ls_results_per_path(parse_input(&example))).for_each_subdirectory(&mut |dir: &Directory| println!("{}", dir.size()));

48381165
24933642
94853
584


In [18]:
fn part1(input: &[String]) -> usize {
    let mut result: usize = 0;
    let root = file_system(ls_results_per_path(parse_input(&input)));
    root.for_each_subdirectory(&mut |dir: &Directory| {
        let size = dir.size();
        if size <= 100000 {
            result += size;
        }
    });
    
    result
}

In [22]:
println!("{}", part1(&example));

95437


In [24]:
part1(&example)

95437

In [19]:
part1(&lines)

1325919

In [29]:
enum DirectoryIterationState<'a> {
    OwnDirectory(&'a Directory),
    Subdirectories(std::collections::hash_map::Values<'a, String, Directory>),
    Finished
}

struct NestedDirectoriesIterator<'a> {
    state: Vec<DirectoryIterationState<'a>>
}

impl NestedDirectoriesIterator<'a> {
    fn next_value(self: &Self) -> Option<&'a Directory> {
        match self.state.last() {
            None => None,
            Some(DirectoryIterationState::OwnDirectory(dir)) => Some(dir),
            DirectoryIterationState::Subdirectories(_) => panic!("after calling advance(), the element of state should not be Subdirectories(_)")
            DirectoryIterationState::Finished => None,
        }
    }
    
    fn advance_last(self: &mut Self) {
        match
    }
}

impl<'a> Iterator for NestedDirectoriesIterator<'a> {
    type Item = &'a Directory;
    
    fn next(self: &mut Self) -> Option<Self::Item> {
        if let Some(current) = self.state.last_mut() {
            match current {
                DirectoryIterationState::OwnDirectory(dir) => {
                    *current = DirectoryIterationState::Subdirectories(dir.directories.values());
                    return Some(dir);
                }
                DirectoryIterationState::Subdirectories(it_subdirectories) => {
                    if let Some(next) = it.subdirectories.next() {
                        
                    }
                }
            }
        } else {
            None
        }
    }
}

Error: non-exhaustive patterns: `&mut DirectoryIterationState::Subdirectories(_)` and `&mut DirectoryIterationState::Finished` not covered

Error: cannot assign to `*current` because it is borrowed

## References
* Slice patterns: https://adventures.michaelfbryan.com/posts/daily/slice-patterns/
* Mofifying value inside `Option<T>`: https://stackoverflow.com/questions/62069793/how-to-edit-values-in-option-by-reference
* Modify last item of a `Vec`: https://doc.rust-lang.org/std/vec/struct.Vec.html#method.last_mut
* https://doc.rust-lang.org/std/string/struct.String.html#method.as_str